In [ ]:
# Dependencies
import json
import requests
from bs4 import BeautifulSoup as bs
import pymongo

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
# Define database and collection
db = client.etl_project_db
collection_oc = db.oc_data
collection_city = db.city_zipcode

In [ ]:
# URL of page to be scraped

# Orange County: https://data-ocpw.opendata.arcgis.com/datasets/d625d46014c44e68a483ab0e74be2aa2_7/data
url_oc = 'https://opendata.arcgis.com/datasets/d625d46014c44e68a483ab0e74be2aa2_7.geojson'

# Los Angeles: https://data.lacity.org/dataset/2010-Census-Populations-by-Zip-Code/nxs9-385f/data
url_la = 'https://data.lacity.org/resource/nxs9-385f.json'

# California Zip Codes: https://www.zip-codes.com/state/ca.asp
url_city_data = 'https://www.zip-codes.com/state/ca.asp'


In [ ]:
# Retrieve page with the requests module
response = requests.get(url_oc).json()
response_city = requests.get(url_city_data)

# Create BeautifulSoup object for scraping California Zip Codes'
soup = bs(response_city.text, 'html.parser')
print(soup)

In [ ]:
# For loop to extract data from JSON for Population Data
for i in range(len(response['features'])):
    zipcode = response['features'][i]["properties"]["GEOID10"]
    total_population = response['features'][i]["properties"]["DP0010001"]
    male_population = response['features'][i]["properties"]["DP0010020"]
    female_population = response['features'][i]["properties"]["DP0010039"]
    total_household = response['features'][i]["properties"]["DP0130001"]
    
    post = {
        'zipcode':zipcode,
        'total_population':total_population,
        'male_population':male_population,
        'female_population':female_population,
        'total_household':total_household
    }
    
    # Dictionary to be inserted as a MongoDB document
    collection_oc.insert_one(post)

In [ ]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
# For California Zip Code
results = soup.find_all('tr')

print(results)

In [ ]:
# For loop to insert data to Mongo for California Zip Code and ignore first column
row_num = 0
for result in results:
    if result.find('td'):
         if result.find_all('a'):
            if row_num>1:
                zipcode = result.find_all('a')[0].text.strip('ZIP Code ')
                city = result.find_all('a')[1].text
                post = {'zipcode':zipcode,
                        'city':city}
                # Dictionary to be inserted as a MongoDB document
                collection_city.insert_one(post)
            row_num = row_num + 1

In [ ]:
# Display items in MongoDB collection
oc_data = db.oc_data.find()

for data in oc_data:
    print(data)